In [1]:
# import libraries
import os
from dotenv import load_dotenv
import datetime
import requests
from bs4 import BeautifulSoup
import pandas as pd
import plotly.express as px
from geopy.distance import geodesic
import googlemaps
import time
from pathlib import Path
from narrow_gate.src.utils.scrape import  *
from narrow_gate.src.utils.finder import  *

In [2]:
Path.cwd()

PosixPath('/mnt/g/Side Projects/the-narrow-gate/narrow-gate')

In [ ]:
# get secrets
# load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')

In [4]:
#set up
gmaps = googlemaps.Client(key=GOOGLE_MAPS_API_KEY)
# genai.configure(api_key=GEMINI_API_KEY)
URL = 'https://www.mass-schedules.com'
header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"}

In [5]:
# refresh_dataset()
churches = pd.read_csv('./narrow_gate/data/churches.csv')
masses = pd.read_csv('./narrow_gate/data/masses.csv')

In [6]:
masses

,Unnamed: 0,day_of_week,schedule,language,comment,church_name,church_address
0,0,6.0,8:00 AM - 9:00 AM,Kapampangan,Livestream,Archdiocesan Shrine of Christ our Lord of the ...,"C. Dayrit St., Lourdes Sur, Angeles City, Pamp..."
1,1,3.0,5:30 PM - 6:30 PM,English,NaN,Archdiocesan Shrine of Christ our Lord of the ...,"C. Dayrit St., Lourdes Sur, Angeles City, Pamp..."
2,2,1.0,5:00 AM - 6:00 AM,Kapampangan,NaN,Archdiocesan Shrine of Christ our Lord of the ...,"C. Dayrit St., Lourdes Sur, Angeles City, Pamp..."
3,3,1.0,6:30 AM - 7:30 AM,Kapampangan,FB Live,Archdiocesan Shrine of Christ our Lord of the ...,"C. Dayrit St., Lourdes Sur, Angeles City, Pamp..."
4,4,1.0,8:00 AM - 9:00 AM,English,FB Live,Archdiocesan Shrine of Christ our Lord of the ...,"C. Dayrit St., Lourdes Sur, Angeles City, Pamp..."
...,...,...,...,...,...,...,...
8934,28,6.0,12:00 PM - 1:00 PM,English,NaN,Zamboanga Cathedral,"La Purisima St., Zamboanga City, Zamboanga del..."
8935,29,6.0,3:00 PM - 4:00 PM,English,NaN,Zamboanga Cathedral,"La Purisima St., Zamboanga City, Zamboanga del..."
8936,30,6.0,4:30 PM - 5:30 PM,Cebuano,NaN,Zamboanga Cathedral,"La Purisima St., Zamboanga City, Zamboanga del..."
8937,31,6.0,6:00 PM - 7:00 PM,English,NaN,Zamboanga Cathedral,"La Purisima St., Zamboanga City, Zamboanga del..."


In [7]:
churches

,Unnamed: 0,church_name,address,telephone number,priest,confession,links,additional info,last update,coords,long,lat
0,0,Archdiocesan Shrine of Christ our Lord of the ...,"C. Dayrit St., Lourdes Sur, Angeles City, Pamp...",(045) 624 0344,NaN,NaN,NaN,NaN,"Sun Jun 4, 2023 12:17PM","{'lat': 15.1416878, 'lng': 120.5909457}",120.590946,15.141688
1,1,Christ the King Parish,"Pulung Cacutud, Angeles City, Pampanga",NaN,NaN,NaN,NaN,NaN,"Sun Jun 4, 2023 12:20PM","{'lat': 15.1614718, 'lng': 120.6113599}",120.611360,15.161472
2,2,Holy Cross Parish,"Sapangbato, Angeles City, Pampanga",0918 963 6828,NaN,NaN,NaN,NaN,"Sun Jun 4, 2023 2:06PM","{'lat': 15.1670159, 'lng': 120.4898564}",120.489856,15.167016
3,3,Holy Rosary Parish (Pisamban Maragul),"Sto. Rosario St., Angeles City, Pampanga",0928 520 7489,NaN,MWF: 6AM-7AM; Sat: 430PM-530PM (except after 1...,NaN,NaN,"Sun Jun 4, 2023 12:28PM","{'lat': 15.1322054, 'lng': 120.5931786}",120.593179,15.132205
4,4,Holy Spirit Parish,"254 Astoria Avenue, Angeles City, Pampanga",0975 045 7696,NaN,NaN,NaN,NaN,"Sun Jun 4, 2023 12:33PM","{'lat': 15.1504415, 'lng': 120.5960318}",120.596032,15.150441
...,...,...,...,...,...,...,...,...,...,...,...,...
676,22,St. Ignatius of Loyola Parish,"Fr. Barua St, Zamboanga City, Zamboanga del Sur",(062) 993-1431,NaN,NaN,NaN,Eucharistic Hour every Thursday at 8PM,"Fri Sep 29, 2023 9:13PM","{'lat': 6.9179516, 'lng': 122.0900869}",122.090087,6.917952
677,23,"St. Joseph, Spouse of the Blessed Virgin Mary ...","Nunez St., Zamboanga City, Zamboanga del Sur",(062) 991 1657,NaN,NaN,NaN,NaN,"Mon May 1, 2023 8:57PM","{'lat': 6.9067735, 'lng': 122.0780309}",122.078031,6.906773
678,24,St. Padre Pio Parish Church and Shrine,"Sunset View, Cabatangan, Zamboanga City, Zambo...",NaN,NaN,NaN,NaN,NaN,"Mon May 1, 2023 10:38PM","{'lat': 6.945071599999999, 'lng': 122.0525894}",122.052589,6.945072
679,25,St. Stanislaus Kostka Parish,"Curuan, Zamboanga City, Zamboanga del Sur",NaN,NaN,NaN,NaN,NaN,"Fri Sep 29, 2023 9:03PM","{'lat': 7.205373899999999, 'lng': 122.2284513}",122.228451,7.205374


In [8]:
sample_address = 'Skyway Twin Towers Condominium Capt. Javier st. Brgy Oranbo Pasig city'
search_range = 10
target_sched = '6:00 PM'
current_datetime = datetime.datetime.now()

In [20]:
church_results, mass_results = find_mass(churches, masses, sample_address, search_range, target_sched, current_datetime)

fig = px.scatter_map(church_results[church_results.church_name.isin(mass_results.church_name)],
                     lat="lat",
                     lon="long",
                     hover_name="church_name", # display church name on hover
                     text="church_name",
                     )
sample_geocode = gmaps.geocode(sample_address)
fig.add_scattermap(lon=[sample_geocode[0]['geometry']['location']['lng']],
                   lat=[sample_geocode[0]['geometry']['location']['lat']],
                   name='my location'
                   )

fig.show()
mass_results

/mnt/g/Side Projects/the-narrow-gate/narrow-gate/narrow_gate/src/utils/finder.py:47: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(57, 13)
6:00 PM
['8:00 AM' '5:30 PM' '5:00 AM' '6:30 AM' '9:30 AM' '11:00 AM' '12:30 PM'
 '2:00 PM' '3:30 PM' '5:00 PM' '6:30 PM' '8:30 AM' '6:00 PM' '7:00 AM'
 '5:30 AM' '10:00 AM' '3:00 PM' '4:30 PM' '7:30 PM' '6:00 AM' '7:00 PM'
 '9:00 AM' '7:30 AM' '4:00 PM' '6:15 AM' '5:15 PM' '7:15 AM' '12:15 PM'
 '10:30 AM' '8:45 AM' '12:00 PM' '1:15 PM' '2:30 PM' '3:45 PM' '6:15 PM'
 '9:45 AM' '4:15 PM' '6:45 PM' '11:30 AM' '1:00 PM' '7:45 AM' '2:45 PM'
 '5:15 AM' '8:30 PM' '6:45 AM' '9:00 PM' '4:15 AM' '5:45 AM' '9:15 AM'
 '8:00 PM' '5:45 PM' '7:45 PM' '4:30 AM' '1:30 PM' '10:45 AM' '7:15 PM'
 '8:15 AM' '2:15 PM' '4:45 PM' '11:45 AM' '4:45 AM' '8:15 PM' '10:15 AM'
 '11:15 AM' '5:50 AM' '4:00 AM' '12:10 PM' '6:25 AM' '7:50 AM' '3:15 PM'
 '12:00 AM']
6:00 PM
(26, 9)


/mnt/g/Side Projects/the-narrow-gate/narrow-gate/narrow_gate/src/utils/finder.py:69: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

/mnt/g/Side Projects/the-narrow-gate/narrow-gate/narrow_gate/src/utils/finder.py:69: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Unnamed: 0,day_of_week,schedule,language,comment,church_name,church_address,mass_start,mass_end,Travel Time(Mins),Arrival Time
6436,11,6.0,6:00 PM - 7:00 PM,NaN,NaN,Sta. Rosa de Lima Parish,"San Buenaventura Drive, Bagong Ilog, Pasig Ci...",6:00 PM,7:00 PM,4.683333,01:51 PM
7084,24,6.0,6:00 PM - 7:00 PM,NaN,NaN,Mary Queen of Peace Shrine (EDSA Shrine),"EDSA cor. Ortigas Ave, Quezon City, Metro Manila",6:00 PM,7:00 PM,8.383333,01:55 PM
3279,25,6.0,6:00 PM - 7:00 PM,NaN,NaN,Mater Dolorosa Parish,"19th Ave., East Rembo, Makati City, Metro Manila",6:00 PM,7:00 PM,10.050000,01:56 PM
8510,4,6.0,6:00 PM - 7:00 PM,NaN,NaN,St. Gabriel the Archangel Chapel,"4th Flr., Uptown Mall, 36th St. corner 9th Ave...",6:00 PM,7:00 PM,10.700000,01:57 PM
3630,9,6.0,6:00 PM - 7:00 PM,NaN,NaN,Sta. Teresita Parish,"A. Mabini St., West Rembo, Makati City, Metro ...",6:00 PM,7:00 PM,11.250000,01:58 PM


In [17]:
church_results 

,Unnamed: 0,church_name,address,telephone number,priest,confession,links,additional info,last update,coords,long,lat,relative_dist,Travel Time(mins)
488,13,Sta. Rosa de Lima Parish,"San Buenaventura Drive, Bagong Ilog, Pasig Ci...",288915129,NaN,Friday - 5:30 PM,NaN,NaN,"Fri Dec 15, 2023 10:55PM","{'lat': 14.5666298, 'lng': 121.0712477}",121.071248,14.566630,0.920683,4.683333
306,25,Sta. Teresita Parish,"A. Mabini St., West Rembo, Makati City, Metro ...",288815241,NaN,NaN,NaN,NaN,"Mon Jun 5, 2023 9:16PM","{'lat': 14.5592198, 'lng': 121.0626459}",121.062646,14.559220,1.664583,11.250000
321,2,Our Lady of Fatima Parish,"Mariveles Street., Highway Hills, Mandaluyong ...",532-8400,NaN,NaN,NaN,NaN,"Wed Mar 17, 2010 10:43AM","{'lat': 14.5814993, 'lng': 121.0502427}",121.050243,14.581499,1.978342,14.216667
302,21,St. John Mary Vianney Parish,"Narra St. , Brgy. Cembo, Makati City, Metro Ma...",288815078,NaN,NaN,NaN,NaN,"Mon Jun 5, 2023 9:25PM","{'lat': 14.5629377, 'lng': 121.0517217}",121.051722,14.562938,2.013209,12.466667
481,6,San Guillermo Parish,"103 E. Mendoza St., Buting, Pasig City, Metro ...",(632) 642-8051,Fr. Bebot Catuiran,30 minutes before the mass; during Holy Hour o...,NaN,NaN,"Sun Oct 20, 2013 1:52PM","{'lat': 14.5550453, 'lng': 121.0669212}",121.066921,14.555045,2.066051,13.350000
288,7,Mater Dolorosa Parish,"19th Ave., East Rembo, Makati City, Metro Manila",757-5704,"Fr. Joel Mosura, TC","Weekdays 5:30 p.m. - 6:00 p.m. , Sat and Sun 5...",NaN,NaN,"Wed Sep 4, 2013 1:46PM","{'lat': 14.5539048, 'lng': 121.0619597}",121.061960,14.553905,2.252680,10.050000
541,32,Mary Queen of Peace Shrine (EDSA Shrine),"EDSA cor. Ortigas Ave, Quezon City, Metro Manila","0998 552 5309, (+632) 8631 5734, 8637 0295",NaN,Upon request,NaN,NaN,"Thu Dec 14, 2023 5:35PM","{'lat': 14.5929003, 'lng': 121.0589506}",121.058951,14.592900,2.283667,8.383333
479,4,San Agustin Parish,"Market Ave., Barangay Palatiw, Pasig City, Met...",286401319,"Rev. Fr. Emmanuel Judilla, O.A.D",NaN,NaN,NaN,"Wed Dec 14, 2022 10:23PM","{'lat': 14.5630345, 'lng': 121.0849293}",121.084929,14.563035,2.286023,12.050000
633,9,St. Gabriel the Archangel Chapel,"4th Flr., Uptown Mall, 36th St. corner 9th Ave...",NaN,NaN,NaN,NaN,Simbang Gabi: December 15-23 6:00PM,"Sun Mar 13, 2022 8:11PM","{'lat': 14.5565479, 'lng': 121.054103}",121.054103,14.556548,2.339422,10.700000
492,17,The Medical City,"Ortigas Avenue, Pasig City, Metro Manila",NaN,NaN,11:00 am daily,NaN,NaN,"Sat May 25, 2013 5:11PM","{'lat': 14.5901598, 'lng': 121.0818858}",121.081886,14.590160,2.441739,12.550000
